## 0. 라이브러리

In [1]:
import os
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import json
from soynlp.normalizer import *
from tqdm import tqdm
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


## 1. 데이터 불러오기

In [79]:
train_data_path ="~/aiffel/dktc/data2/train1.csv"
train_data = pd.read_csv(train_data_path,index_col=0)
train_data


,class,conversation
idx,,
1,일반 대화,오늘 날씨 어때?\n맑고 따뜻해.\n좋네! 주말에 계획 있어?\n등산 갈 생각이야....
2,일반 대화,오늘 날씨 어때?\n맑고 따뜻해.\n좋네! 주말에 계획 있어?\n등산 갈 생각이야....
3,기타 괴롭힘 대화,너 피아노 좀 쳐봐.\n싫어.\n왜 손가락 없다고 유세 떠는 거야?\n이씨.\n비행...
4,갈취 대화,요즘에는 액수가 작네?\n미안해 요즘에 용돈이 작아\n그게 나랑 무슨 상관이야?\n...
5,기타 괴롭힘 대화,야 이거봐 완전 길동이 닯음\n 진짜네 \n야 그러지마.\n왜 똑같구만 원숭이 \n...
...,...,...
4946,직장 내 괴롭힘 대화,어이 신병.\n이병 김범례.\n와봐.\n네.\n네? 뒤질래?\n.자.잘못들었습니다?...
4947,직장 내 괴롭힘 대화,오늘 회의 안건인 길동프로그램의 출연자는 누구로 할 것인가에 대해 모두 의견 내주시...
4948,직장 내 괴롭힘 대화,야 열심히들 해라 새끼들아\n넵 감사해요 부장님\n야 내가 언제 너 한테 말했냐\n...


## 2. 데이터 준비 (Data preparation)
### 2.1-1 전처리 함수 정의

In [3]:
def preprocess_sentence(sentence):
    emoticon_normalize(sentence)
    repeat_normalize(sentence)
    sentence = re.sub(r'([^a-zA-Zㄱ-ㅎ가-힣?.!,])', " ", sentence)
    sentence = re.sub(r'!+', '!', sentence)
    sentence = re.sub(r'\?+', '?', sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence


### 2.1-2 전처리 함수 적용

In [4]:
# 학습할 문장이 담길 배열
sentences = []

for val in tqdm(train_data['conversation']):
    sentences.append(preprocess_sentence(val))


100%|██████████| 4950/4950 [00:01<00:00, 3301.16it/s]


### 2.2 최대 길이 지정

In [5]:
MAX_LEN = 200


### 2.3 class(label) 인코딩

In [6]:
from sklearn.preprocessing import LabelEncoder

CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화','일반 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])
labels = train_data['class']

len(labels)


4950

In [ ]:
class_mapping = {class_name: encoder.transform([class_name])[0] for class_name in CLASS_NAMES}
print("Class mapping:", class_mapping)


### 2.4 train-val

In [7]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)


## 3. 모델
### 3.1-1 토크나이저 정의

In [8]:
# BERT 토크나이저와 모델 준비
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")


### 3.1-2 토크나이저 적용

In [9]:
# 데이터셋을 BERT 입력 형식으로 변환
train_encodings = tokenizer(
    train_sentences, truncation=True, padding=True, max_length=MAX_LEN
)  # 뒤쪽에 패딩
val_encodings = tokenizer(
    val_sentences, truncation=True, padding=True, max_length=MAX_LEN
)


### 3.2 모델 준비

In [10]:
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", num_labels=5
)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 3.3 파라미터

In [11]:
BATCH_SIZE = 16
lr = 5e-5
EPOCH = 10


### 3.4 TF 데이터셋 생성

In [ ]:
# TensorFlow 데이터셋 생성
train_dataset = (
    tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
    .shuffle(100)
    .batch(BATCH_SIZE)
)

val_dataset = tf.data.Dataset.from_tensor_slices(
    (dict(val_encodings), val_labels)
).batch(BATCH_SIZE)


### 3.5 모델 컴파일

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


### 3.6 모델 훈련

### 3.6-1 콜백 설정

In [20]:
early_stopping = EarlyStopping(
    monitor='val_loss',    # 검증 손실을 모니터링
    patience=3,            # 3 에포크 동안 개선되지 않으면 중지
    restore_best_weights=True  # 최상의 가중치를 복원
)

checkpoint = ModelCheckpoint(
    filepath='best_model_weights.h5',  # 모델 가중치를 저장할 파일 경로
    monitor='val_loss',        # 검증 손실을 모니터링
    save_best_only=True,       # 최상의 모델만 저장
    save_weights_only=True,   # 저장 (가중치)
    mode='min',                # 'val_loss'가 최소일 때 저장
    verbose=1                  # 저장 시 로그 출력
)


### 3.6-2 모델 훈련

In [21]:
model.fit(
    train_dataset, 
    validation_data=val_dataset,
    epochs=EPOCH,
    callbacks=[early_stopping, checkpoint]
)


Epoch 1/10
248/248 [==============================] - 186s 750ms/step - loss: 0.5683 - accuracy: 0.8005 - val_loss: 0.5668 - val_accuracy: 0.8000

Epoch 00001: val_loss improved from inf to 0.56680, saving model to best_model_weights.h5
Epoch 2/10
248/248 [==============================] - 186s 751ms/step - loss: 0.4551 - accuracy: 0.8414 - val_loss: 0.4392 - val_accuracy: 0.8465

Epoch 00002: val_loss improved from 0.56680 to 0.43918, saving model to best_model_weights.h5
Epoch 3/10
248/248 [==============================] - 186s 752ms/step - loss: 0.3632 - accuracy: 0.8775 - val_loss: 0.4361 - val_accuracy: 0.8444

Epoch 00003: val_loss improved from 0.43918 to 0.43609, saving model to best_model_weights.h5
Epoch 4/10
248/248 [==============================] - 186s 751ms/step - loss: 0.2754 - accuracy: 0.9139 - val_loss: 0.4817 - val_accuracy: 0.8505

Epoch 00004: val_loss did not improve from 0.43609
Epoch 5/10
248/248 [==============================] - 186s 750ms/step - loss: 0.212

### 3.7 모델 평가

In [22]:
# 모델 평가
evaluation = model.evaluate(val_dataset)
print("평가 결과:", evaluation)


62/62 [==============================] - 14s 217ms/step - loss: 0.4361 - accuracy: 0.8444
평가 결과: [0.4360905885696411, 0.8444444537162781]


## 4. 모델 적용

In [80]:
test_data_path = "~/aiffel/dktc/data2/test.json"

with open(test_data_path, "r", encoding="utf-8") as json_file:
    test = json.load(json_file)


FileNotFoundError: [Errno 2] No such file or directory: '~/aiffel/dktc/data2/test.json'

In [35]:
import numpy as np

test_predicst = list()

for key in test:
    test_sentence = test['text']
    
    test_encodings = tokenizer(test_sentence, truncation=True, padding=True, max_length=128, return_tensors="tf")
    
    test_predictions = model.predict({
        "input_ids": test_encodings["input_ids"],
        "token_type_ids": test_encodings["token_type_ids"],
        "attention_mask": test_encodings["attention_mask"]
    }) # [ 0.7805823,  2.6188664, -2.0281641, -0.9672525]
    test_class_probabilities = tf.nn.softmax(test_predictions.logits, axis=-1).numpy() # [[0.13297564 0.8358507  0.00801584 0.02315779]]
    test_predicted_class = np.argmax(test_class_probabilities, axis=1) # [ 1 ]
    test_predicst.append(test_predicted_class[0])


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
def labelnum_to_text(x):
    if x == 1 : # 갈취
        return '01'
    if x == 2 : # 직장
        return '02'
    if x == 3 : # 기타
        return '03'
    if x == 0 : # 협박 
        return '00'
    
submission = pd.DataFrame({'class':test_predicst}, index=list(test.keys()))

submission['class'] = submission['class'].apply(labelnum_to_text)
submission


In [ ]:
#  submission.to_csv('~/aiffel/dktc/sub/base_sub.csv')
